# Importar Modulos

In [19]:
import pandas as pd
import numpy as np
import random
import plotly.express as px
import tensorflow as tf

from sklearn.utils import shuffle
import tensorflow_addons as tfa
from IPython.display import display

# Importación de Datos

In [20]:
#el conj de datos va de 0 a 2623 o sea 2624 datos
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format

# Recompila la información de un archivo CSV y lo guarda en un arreglo
data_copy = np.loadtxt("lp5.csv", delimiter=",", dtype=str)
#  Contador de filas en el CSV
cont = 16
labels = []
numbers = []
# Arreglo de ceros con las dimensiones de los datos
features= np.zeros((164,15,6))
test_features= np.zeros((164,15,6))

# Ciclo que recorre todos los datos del CSV y guarda los títulos de los datos cuando el contador es 16,
# guarda los números en un arreglo aparte y reestablece el contador a su valor original cuando es igual a 0.
for i in range(len(data_copy)):
    if(cont == 16):
        labels.append(data_copy[i])
    if(cont<16):
        numbers.append(data_copy[i])
    cont -= 1
    if(cont == 0):
        cont = 16
        
test_numbers = np.array(numbers).astype(float)

for i in range(len(test_numbers)):
    for j in range(5):
        test_numbers[i][j] = test_numbers[i][j] + test_numbers[i][j]*random.uniform(-0.05, 0.05)

display(test_numbers)
# Ciclos anidados que recorren todas las dimensiones del arreglo de ceros,
# agrupa los datos numéricos en conjuntos con forma 15x6 
cont = 0
for i in range(164):
    for j in range(15):
        for z in range(6):
            features[i][j][z] = numbers[cont][z]
            test_features[i][j][z]= test_numbers[cont][z]
        cont += 1
        
# Convierte los datos obtenidos del primer ciclo en un dataframe
labels_df = pd.DataFrame(labels, columns=['labels','1','2','3','4','5'])

display(labels_df)
display(features)
display(test_features)

array([[-1.90091157, -0.95649244, 84.661783  ,  0.        , -5.14111195,
         0.        ],
       [-1.97916995, -1.02859692, 80.6861193 ,  0.        , -3.94172726,
         0.        ],
       [-2.06908788, -0.95496617, 76.42199044,  0.        , -4.0400594 ,
         0.        ],
       ...,
       [-0.97407015,  0.96271487, -2.86648999, -3.13154504, -1.92349804,
        -3.        ],
       [ 0.        , -1.01153882, -4.94835311, -1.00633636,  0.95668863,
        -3.        ],
       [-1.02420578,  0.97435674,  4.1365242 ,  0.        , -1.00943195,
        -3.        ]])

,labels,1,2,3,4,5
0,normal,,,,,
1,normal,,,,,
2,normal,,,,,
3,normal,,,,,
4,normal,,,,,
...,...,...,...,...,...,...
159,collision_in_tool,,,,,
160,collision_in_tool,,,,,
161,collision_in_tool,,,,,
162,collision_in_tool,,,,,


array([[[ -2.,  -1.,  81.,   0.,  -5.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        ...,
        [ -2.,  -1.,  78.,   0.,  -5.,   0.],
        [ -3.,  -1.,  80.,   1.,  -4.,   1.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.]],

       [[  6.,  -1.,  79.,  -2.,   4.,  -3.],
        [ 42.,  -3.,  80.,   5.,  53.,   3.],
        [ -5.,   4.,  74., -15., -10.,  -1.],
        ...,
        [ -1.,  -5.,  80.,   6.,  -6.,   0.],
        [ -4.,   5.,  78., -14.,  -9.,  -4.],
        [ -4.,   1.,  80.,  -3., -12.,   5.]],

       [[ -2.,  -6.,  85.,  14.,  -5.,   2.],
        [  0.,   2.,  74.,  -7.,   1.,   0.],
        [ -4.,  -5.,  76.,   7., -11.,   4.],
        ...,
        [  0.,  -9.,  87.,  13.,  -5.,   2.],
        [ -5.,   5.,  67., -17., -16.,   7.],
        [ -6., -10.,  86.,  16., -14.,  -1.]],

       ...,

       [[-12.,  17.,   3., -19., -10.,  -4.],
        [-12.,  12.,  11., -13., -16.,  -4.],
        [ -8.,   3.,  

array([[[ -1.90091157,  -0.95649244,  84.661783  ,   0.        ,
          -5.14111195,   0.        ],
        [ -1.97916995,  -1.02859692,  80.6861193 ,   0.        ,
          -3.94172726,   0.        ],
        [ -2.06908788,  -0.95496617,  76.42199044,   0.        ,
          -4.0400594 ,   0.        ],
        ...,
        [ -2.05654688,  -0.99667058,  74.60197674,   0.        ,
          -4.93022873,   0.        ],
        [ -3.05843784,  -0.9744253 ,  80.73856599,   0.9645761 ,
          -3.81339313,   1.        ],
        [ -2.02688314,  -1.00187858,  78.17440953,   0.        ,
          -4.06688821,   0.        ]],

       [[  5.99046207,  -1.01732533,  79.94620355,  -2.02683152,
           3.93532637,  -3.        ],
        [ 42.46353058,  -3.04216732,  81.51809779,   5.10371105,
          54.0579936 ,   3.        ],
        [ -4.78064892,   3.83877091,  75.99069691, -14.92716389,
          -9.89938362,  -1.        ],
        ...,
        [ -1.0188209 ,  -4.86303271,  81.2693

# Manejo de Datos y Normalización

In [21]:
# Diccionario que codifica los datos categóricos a un conjunto de números idóneo para la capa de salida.
classes_dict = {'normal':'1 0 0 0 0', #1
        'collision_in_tool':'0 1 0 0 0', #2
        'collision_in_part':'0 0 1 0 0', #3
        'bottom_collision':'0 0 0 1 0', #4
        'bottom_obstruction':'0 0 0 0 1'} #5

# Se reemplazan los datos por lo del diccionario
labels_df = labels_df.replace({'labels':classes_dict})
# Se separan los datos de manera que se aisle un caractér por columna.
labels_df[['1', '2', '3', '4', '5']] = labels_df['labels'].str.split(' ', 4, expand= True)
# Se eliminan las columnas, solo dejando las columnas llamadas "1", "2", "3", "4", "5"
labels_df = labels_df.loc[:,['1', '2', '3', '4', '5']]
# Se convierten los valores de un tipo String a uno Entero para poder ser utilizados por la red neuronal.
labels_df[['1', '2', '3', '4', '5']]=labels_df[['1', '2', '3', '4', '5']].astype(str).astype(int)

'''/* 
Function: norm

Normaliza los datos a través de una función min-max entre 0 y 1 para ser alimentados a la red.

Parameters:

    x - Datos a normalizar.
    
Returns:

    Los datos alimentados ya normalizados.
*/'''

def norm(x):
    x_min = x.min()
    x_max = x.max()
    range = x_max - x_min  #min max entre 0 y 1
    return((x-x_min)/(range))

# Normaliza las características.
train_features = norm(features)
test_features = norm(test_features)
# Split para entrenamiento y validacion, con 20% para testeo y se aleatorizan.
train_labels = labels_df
test_labels = labels_df

train_features, train_labels = shuffle(train_features, train_labels, random_state=0)
test_features, test_labels = shuffle(test_features, test_labels, random_state= 0)
display(test_features)
display(test_labels)

C:\Users\dave4\AppData\Local\Temp\ipykernel_9460\1604112957.py:11: FutureWarning:

In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.



array([[[0.78502244, 0.78242509, 0.77006613, 0.77503371, 0.78618342,
         0.7847806 ],
        [0.7869804 , 0.78271496, 0.75983525, 0.77320042, 0.79319854,
         0.78330158],
        [0.7867449 , 0.78217168, 0.74608883, 0.77439453, 0.79866353,
         0.77992098],
        ...,
        [0.78264288, 0.78455888, 0.78270316, 0.78139171, 0.7824454 ,
         0.78393545],
        [0.78352295, 0.78413043, 0.78263191, 0.78197864, 0.78351715,
         0.78393545],
        [0.78394065, 0.78350366, 0.78269838, 0.78287651, 0.78416222,
         0.78414673]],

       [[0.77290472, 0.78700998, 0.7801035 , 0.77588944, 0.76262297,
         0.77907583],
        [0.77350486, 0.78774615, 0.78536657, 0.77727354, 0.77077461,
         0.78435802],
        [0.77503557, 0.78786614, 0.78638597, 0.777891  , 0.77221254,
         0.78330158],
        ...,
        [0.7845952 , 0.78204781, 0.75037446, 0.78558633, 0.78872161,
         0.78266772],
        [0.78459181, 0.78174561, 0.74752262, 0.78570908, 0.788

,1,2,3,4,5
146,0,0,0,1,0
89,0,0,1,0,0
106,0,0,0,1,0
7,1,0,0,0,0
94,0,1,0,0,0
...,...,...,...,...,...
9,1,0,0,0,0
103,0,0,0,0,1
67,1,0,0,0,0
117,0,0,0,0,1


# Creación del Modelo

In [22]:
'''/* 
Function: my_model

Genera el modelo categórico convolucional y lo compila. El modelo consiste de lo siguiente:
    - Una capa de entradas de forma (15,6,1) que es igual a las dimensiones de la matriz de datos.
    
    - Una capa de convolución de dos dimensiones que genera 8 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de convolución de dos dimensiones que genera 8 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de Max Pooling con un pool de 2x2, un salto de 2 casillas y sin padding.
    - Una capa de Dropout razón con una razón de 0.3

    - Una capa de convolución de dos dimensiones que genera 16 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de convolución de dos dimensiones que genera 16 filtros, tiene un kernel 3x3, 
        función de activación ReLu y un padding para que mantenga las dimensiones de la entrada.
    - Una capa de Max Pooling con un pool de 2x2, un salto de 2 casillas y sin padding.
    - Una capa de Dropout razón con una razón de 0.2.
    
    - Una capa de flatten para pasar de varias dimensiones a una.
    - Una capa oculta con 24 neuronas y con una función de activación ReLu.
    - Una capa de salida con 5 neuronas. Una por categoría.
    
    - Utiliza ADAM como optimizador.
    - La función de pérdida es la Entropía Cruzada Categórica.
    - La métrica de evaluación es la exactitud.
    
Parameters:

    my_learning_rate - Razón de aprendizaje.
    
Returns:

    El modelo neuronal ya creado.
*/'''

def my_model(learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape = (15,6,1), padding='same'), # #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding= 'valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),# #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding='valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model

model = model = my_model(0.01)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 15, 6, 8)          80        
                                                                 
 conv2d_21 (Conv2D)          (None, 15, 6, 8)          584       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 7, 3, 8)          0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 7, 3, 8)           0         
                                                                 
 conv2d_22 (Conv2D)          (None, 7, 3, 16)          1168      
                                                                 
 conv2d_23 (Conv2D)          (None, 7, 3, 16)          2320      
                                                      

# Entrenamiento del Modelo

In [23]:
'''/* 
Function: train_model

Alimenta los datos correspondientes a las características y etiquetas al modelo y conduce el proceso de entrenamiento y validación.
Además de esto, también le establece al modelo la cantidad de ciclos de entrenamiento y el tamaño del batch.
Por último crea un subconjunto de validación con un 25% de los datos de entrenamiento.
    
Parameters:

    model - El modelo creado anteriormente.
    features - Conjunto que contiene las características.
    labels - Conjunto que contiene las etiquetas.
    epochs - Cantidad de ciclos de entrenamiento
    batch_size - Tamaño del batch.
    
Returns:

    Los resultados del entrenamiento.
*/'''


def train_model(model, features, labels, epochs, batch_size):
    history = model.fit(
        x = features,
        y = labels,
        epochs= epochs,
        batch_size= batch_size,
        validation_split= 0.25
    )
    # Guarda los resultados obtenidos del proceso de entrenamiento y validación en un DataFrame.
    # Estos resultados son las pérdidas y el número del ciclo correspondiente.
    hist= pd.DataFrame(history.history)
    # Añade al DataFrame la información sobre los ciclos.
    hist['epoch'] = history.epoch
    
    return hist

# Visualizaciones

In [24]:
'''/* 
Function: loss_curve

Grafica las curvas de pérdida correspondientes al entrenamiento y la validación.
    
Parameters:

    history - Resultados provenientes del proceso de entrenamiento.
    
Returns:

    Las gráficas ya creadas.
*/'''

def loss_curves(history):
    hist = history
    # Cambia los títulos de cada columna que contiene los datos de exactitud por una versión más legible
    labels = {"loss":"Training Loss", "val_loss":"Validation Loss"}
    hist.rename(columns = labels, inplace = True)
    
    # Crea la figura, establece los títulos de eje y la paleta de colors
    fig = px.line(hist, x='epoch', y=['Training Loss', 'Validation Loss'],
                title='Gráficas de Pérdida de Entrenamiento y Evaluación',
                labels={"epoch": "Epoch", "value":"Binary Cross Entropy", "variable":"Curvas de Pérdida"},
                color_discrete_map={
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()
    
'''/* 
Function: accuracy_curve

Grafica las curvas de exactitud correspondientes al entrenamiento y la validación.
    
Parameters:

    history - Resultados provenientes del proceso de entrenamiento.
    
Returns:

    Las gráficas ya creadas.
*/'''
    
def accuracy_curve(history):
    hist = history
    # Cambia los títulos de cada columna que contiene los datos de exactitud por una versión más legible
    labels = {"val_accuracy":"Exactitud de Validación", "accuracy":"Exactitud de Entrenamiento"}
    hist.rename(columns = labels, inplace = True)
    
    # Crea la figura, establece los títulos de eje y la paleta de colors
    fig = px.line(hist, x='epoch', y=['Exactitud de Entrenamiento', 'Exactitud de Validación'],
                title='Gráficas de Exactitud',
                labels={"epoch": "Epoch", "value":"Exactitud", "variable":"Curvas de Exactitud"},
                color_discrete_map={
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()

# Se corren las funciones

In [25]:
# Hiperparámetros
learning_rate = 0.001
epochs = 350
batch_size = 5
# Llama a la función para crear el modelo y lo guarda.
model = my_model(learning_rate)
# Invoca a la función de entrenamiento y guarda los resultados.
history= train_model(model, train_features, train_labels, epochs, batch_size)
# Llama a la función de las gráficas.
loss_curves(history)
accuracy_curve(history)
display(history)

Epoch 1/350


C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5561: UserWarning:

"`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?



25/25 [==============================] - 1s 8ms/step - loss: 1.6016 - accuracy: 0.2683 - val_loss: 1.5952 - val_accuracy: 0.1951
Epoch 2/350
25/25 [==============================] - 0s 3ms/step - loss: 1.5812 - accuracy: 0.2927 - val_loss: 1.5708 - val_accuracy: 0.1951
Epoch 3/350


C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5561: UserWarning:

"`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?



25/25 [==============================] - 0s 4ms/step - loss: 1.5881 - accuracy: 0.2846 - val_loss: 1.5701 - val_accuracy: 0.1951
Epoch 4/350
25/25 [==============================] - 0s 4ms/step - loss: 1.5694 - accuracy: 0.2927 - val_loss: 1.5676 - val_accuracy: 0.1951
Epoch 5/350
25/25 [==============================] - 0s 4ms/step - loss: 1.5622 - accuracy: 0.2927 - val_loss: 1.5666 - val_accuracy: 0.1951
Epoch 6/350
25/25 [==============================] - 0s 4ms/step - loss: 1.5689 - accuracy: 0.3008 - val_loss: 1.5679 - val_accuracy: 0.1951
Epoch 7/350
25/25 [==============================] - 0s 4ms/step - loss: 1.5592 - accuracy: 0.2846 - val_loss: 1.5564 - val_accuracy: 0.3659
Epoch 8/350
25/25 [==============================] - 0s 4ms/step - loss: 1.5764 - accuracy: 0.2358 - val_loss: 1.5516 - val_accuracy: 0.1951
Epoch 9/350
25/25 [==============================] - 0s 4ms/step - loss: 1.5796 - accuracy: 0.2764 - val_loss: 1.5621 - val_accuracy: 0.1951
Epoch 10/350
25/25 [=====

,Training Loss,Exactitud de Entrenamiento,Validation Loss,Exactitud de Validación,epoch
0,1.602,0.268,1.595,0.195,0
1,1.581,0.293,1.571,0.195,1
2,1.588,0.285,1.570,0.195,2
3,1.569,0.293,1.568,0.195,3
4,1.562,0.293,1.567,0.195,4
...,...,...,...,...,...
345,0.719,0.691,0.791,0.610,345
346,0.651,0.724,0.812,0.585,346
347,0.613,0.740,0.889,0.585,347
348,0.695,0.740,0.787,0.610,348


# Predicciones

In [26]:
# Hace predicciones usando el conjunto de datos de prueba.
pd.options.display.float_format = "{:.0f}".format
predictions = model.predict(test_features)
for i in range(len(predictions)):
    if(predictions[i].max() != 1):
        row, col = np.where(predictions == predictions[i].max())
        predictions[i][col[0]] = 1
predictions[predictions!=1] = 0 
    
# Convierte el arreglo de predicciones en un DataFrame
predictions_df = pd.DataFrame(predictions, columns=['normal', 'collision_in_tool', 'collision_in_part', 'bottom_collision', 'bottom_obstruction'])
display(predictions_df)
display(test_labels)
CM = tfa.metrics.MultiLabelConfusionMatrix(num_classes=5)
CM.update_state(test_labels, predictions_df)
result = CM.result().numpy()
display(result)

normal = px.imshow(result[0], 
                labels = dict(x='normal (Predicted)', y='normal'),
                x = ['Negative', '-Positive'], y=['True', 'False'],text_auto=True, color_continuous_scale='greys')
normal.update_xaxes(side="top")
normal.show()

collision_in_tool = px.imshow(result[0], 
                labels = dict(x='collision_in_tool (Predicted)', y='collision_in_tool'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
collision_in_tool.update_xaxes(side="top")
collision_in_tool.show()

collision_in_part = px.imshow(result[0], 
                labels = dict(x='collision_in_part (Predicted)', y='collision_in_part'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
collision_in_part.update_xaxes(side="top")
collision_in_part.show()

bottom_collision = px.imshow(result[0], 
                labels = dict(x='bottom_collision (Predicted)', y='bottom_collision'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
bottom_collision.update_xaxes(side="top")
bottom_collision.show()

bottom_obstruction = px.imshow(result[0], 
                labels = dict(x='bottom_obstruction (Predicted)', y='bottom_obstruction'),
                x = ['Negative', '-Positive'], y=['True', 'False'], text_auto=True, color_continuous_scale='greys')
bottom_obstruction.update_xaxes(side="top")
bottom_obstruction.show()
#model.save('ModeloConvolucion_A0.75')

6/6 [==============================] - 0s 2ms/step


,normal,collision_in_tool,collision_in_part,bottom_collision,bottom_obstruction
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,1,0,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
159,1,0,0,0,0
160,0,0,0,0,1
161,1,0,0,0,0
162,0,0,0,0,1


,1,2,3,4,5
146,0,0,0,1,0
89,0,0,1,0,0
106,0,0,0,1,0
7,1,0,0,0,0
94,0,1,0,0,0
...,...,...,...,...,...
9,1,0,0,0,0
103,0,0,0,0,1
67,1,0,0,0,0
117,0,0,0,0,1


array([[[ 85.,  35.],
        [  0.,  44.]],

       [[129.,   9.],
        [ 17.,   9.]],

       [[109.,   8.],
        [ 34.,  13.]],

       [[132.,   6.],
        [  3.,  23.]],

       [[143.,   0.],
        [  4.,  17.]]], dtype=float32)